In [1]:
# Scenario Generation with Copulas 
# 
# Hugo S. de Araujo
# Nov. 14th, 2022 | Mays Group | Cornell University
################################################################################

#=======================================================================
PROJECT SETUP
=======================================================================#
using Pkg
Pkg.activate("copulas");
Pkg.instantiate();
# Import "here" function. Wrapper to allow easy path concatenation.
include(joinpath(@__DIR__, "functions", "fct_here.jl"))

# Import all required packages. 
begin
    # using AWSS3
    using CSV
    using DataFrames
    using Dates
    using DelimitedFiles
    using Distributions
    using HDF5
    using JuliaFormatter
    using LaTeXStrings
    using LinearAlgebra
    using LinearSolve
    #using Measures
    using Random
    using RCall
    using Revise
    using Statistics
    using StatsBase
    #using StatsPlots
    using OhMyREPL
    using Plots
    #using PrettyTables
    using Tables
    using TSFrames
    using TimeZones
end

# Include functions 
#= functions_dirpath = joinpath(pwd(),"src", "functions");
function_paths = readdir(functions_dirpath, join=true);
function_index = occursin.(".jl", function_paths);
functions_only = function_paths[function_index];

for str in functions_only
    include(str)
end =#

include(here("src", "functions", "fct_bind_historical_forecast.jl"));
include(here("src", "functions", "fct_compute_hourly_average_actuals.jl"));
include(here("src", "functions", "fct_compute_landing_probability.jl"));
include(here("src", "functions", "fct_convert_hours_2018.jl"));
include(here("src", "functions", "fct_convert_ISO_standard.jl"));
include(here("src", "functions", "fct_convert_land_prob_to_data.jl"));
include(here("src", "functions", "fct_generate_probability_scenarios.jl"));
include(here("src", "functions", "fct_getplots.jl"));
include(here("src", "functions", "fct_plot_correlation_heatmap.jl"));
include(here("src", "functions", "fct_plot_historical_landing.jl"));
include(here("src", "functions", "fct_plot_historical_synthetic_autocorrelation.jl"));
include(here("src", "functions", "fct_plot_correlogram_landing_probability.jl"));
include(here("src", "functions", "fct_plot_scenarios_and_actual.jl"));
include(here("src", "functions", "fct_read_h5_file.jl"));
include(here("src", "functions", "fct_read_input_file.jl"));
include(here("src", "functions", "fct_transform_landing_probability.jl"));
include(here("src", "functions", "fct_write_percentiles.jl"));
#=======================================================================
READ INPUT FILE
=======================================================================#
input_file_path = here("src\\copulas.txt")

# XXX Needs to be updated to be a hardcoded instead of reading in a text file
data_type,
scenario_length,
number_of_scenarios,
scenario_hour,
scenario_day,
scenario_month,
scenario_year,
read_locally,
historical_load,
forecast_load,
historical_solar,
forecast_da_solar,
forecast_2da_solar,
historical_wind,
forecastd_da_wind,
forecast_2da_wind,
write_percentile = read_input_file(input_file_path);

#=======================================================================
READ INPUT DATA: ARPA-E PERFORM PROJECT H5 FILES
=======================================================================#
# Function that reads the .h5 file and binds the time index and the actuals/fore-
# cast values into a single dataframe.

# Load data
load_actuals_raw = read_h5_file(here("data", historical_load), "load");
load_forecast_raw = read_h5_file(here("data", "ercot_BA_load_forecast_day_ahead_2018.h5"), "load", false);

# Solar data
solar_actuals_raw = read_h5_file(here("data", "ercot_BA_solar_actuals_Existing_2018.h5"), "solar");
solar_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_day_ahead_existing_2018.h5"), "solar", false);
solar_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_2_day_ahead_existing_2018.h5"), "solar", false);

# Wind data
wind_actuals_raw = read_h5_file(here("data", "ercot_BA_wind_actuals_Existing_2018.h5"), "wind");
wind_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_day_ahead_existing_2018.h5"), "wind", false);
wind_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_2_day_ahead_existing_2018.h5"), "wind", false);

#=======================================================================
Compute the hourly average for the actuals data
=======================================================================#
# Load
aux = compute_hourly_average_actuals(load_actuals_raw);
load_actual_avg_raw = DataFrame();
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
load_actual_avg_raw[!, :time_index] = time_index;
load_actual_avg_raw[!, :avg_actual] = avg_actual;

# Solar
aux = compute_hourly_average_actuals(solar_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
solar_actual_avg_raw = DataFrame();
solar_actual_avg_raw[!, :time_index] = time_index;
solar_actual_avg_raw[!, :avg_actual] = avg_actual;

# Wind
aux = compute_hourly_average_actuals(wind_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
wind_actual_avg_raw = DataFrame();
wind_actual_avg_raw[!, :time_index] = time_index;
wind_actual_avg_raw[!, :avg_actual] = avg_actual;

  Activating project at `c:\Users\ks885\Documents\aa_research\Modeling\norta_scenarios\copulas\src\copulas`


In [2]:
# create a function to do this for all data
function upd_convert_hours_2018(data, is_actual = true)
    if is_actual
        x = copy(data);
        x[:,:time_index] = x[:,:time_index] .- Dates.Hour(6);
        return x;
    else
        df = copy(data);
        df[:,:forecast_time] = df[:,:forecast_time] .- Dates.Hour(6);
        df[:,:issue_time] = df[:,:issue_time] .- Dates.Hour(6);
        return df;
    end
end

upd_convert_hours_2018 (generic function with 2 methods)

In [3]:
# Load data
load_actuals = upd_convert_hours_2018(load_actuals_raw);
load_actual_avg = upd_convert_hours_2018(load_actual_avg_raw);
load_forecast = upd_convert_hours_2018(load_forecast_raw, false);

# Solar data
solar_actuals = upd_convert_hours_2018(solar_actuals_raw);
solar_actual_avg = upd_convert_hours_2018(solar_actual_avg_raw);
solar_forecast_dayahead = upd_convert_hours_2018(solar_forecast_dayahead_raw, false);
solar_forecast_2dayahead = upd_convert_hours_2018(solar_forecast_2dayahead_raw, false);

# Wind data
wind_actuals = upd_convert_hours_2018(wind_actuals_raw);
wind_actual_avg = upd_convert_hours_2018(wind_actual_avg_raw);
wind_forecast_dayahead = upd_convert_hours_2018(wind_forecast_dayahead_raw, false);
wind_forecast_2dayahead = upd_convert_hours_2018(wind_forecast_2dayahead_raw, false);


In [4]:
#=======================================================================
BIND HOURLY HISTORICAL DATA WITH FORECAST DATA
========================================================================#
#= The binding is made by ("forecast_time" = "time_index"). This causes the 
average actual value to be duplicated, which is desired, given the # of rows
in the load_forecast is double that of load_actual. To distinguish a 
one-day-ahead forecast from a two-day-ahead forecast, the column "ahead_factor"
is introduced. Bind the day-ahead and two-day-ahead forecasts for wind and solar
to get all the forecast data into one object as it is for load forecast =#
    load_data = bind_historical_forecast(true,
    load_actual_avg,
    load_forecast);

solar_data = bind_historical_forecast(false,
    solar_actual_avg,
    solar_forecast_dayahead,
    solar_forecast_2dayahead);

wind_data = bind_historical_forecast(false,
    wind_actual_avg,
    wind_forecast_dayahead,
    wind_forecast_2dayahead);

In [5]:
#=======================================================================
Landing probability
=======================================================================#
#= This section holds the calculation of the probability that the actual
value was equaled or superior than the forecast percentiles for a given
day. This is made possible by the estimation of an approximate CDF
computed on the forecast percentiles. Once estimated, this function is
used to find the "landing probability"; the prob. that the actual value
is equal or greater than a % percentage of the forecast percentile.
=#
#include(here("src", "functions", "fct_compute_landing_probability.jl"))
landing_probability_load = compute_landing_probability(load_data);
landing_probability_solar = compute_landing_probability(solar_data);
landing_probability_wind = compute_landing_probability(wind_data);

In [6]:
#=======================================================================
ADJUST LANDING PROBABILITY DATAFRAME
=======================================================================#
lp_load = transform_landing_probability(landing_probability_load);
lp_solar = transform_landing_probability(landing_probability_solar);
lp_wind = transform_landing_probability(landing_probability_wind);

In [30]:
lp_solar

363×48 transpose(::Matrix{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱                 ⋮         
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1

Now we get into to the policy loop code

Actually there is some stuff in the generate probability scenarios that needs to be out of loop

In [8]:
data = lp_load;


In [9]:
x = copy(data)
allequal_set = Set(findall(allequal, eachcol(x)));
allequal_ind = sort(collect(allequal_set));
allindex_set = Set(collect(1:48));
alldifferent_ind = sort(collect(setdiff(allindex_set, allequal_set))); # Index for columns whose st. dev. isn't zero
x = x[:, alldifferent_ind];


# ==================================================================
# COLUMNS TO KEEP
# ==================================================================
#=Here, we care only about the columns in x::DataFrame whose elements 
are not all equal. If they are, the correlation is not defined b/c
the standard deviation will be zero for columns whose elements
are all the same =#
if ishermitian(cor(x))
    Σ_Z = LinearAlgebra.cholesky(cor(x));
else
    Σ_Z = factorize(cor(x));
end
M = Σ_Z.L;
dim_M = size(M, 1);

# ==================================================================
# CORRELATION MATRIX
# ==================================================================
#= Determine a lower-triangular, nonsingular factorization M of the 
    the correlation matrix for Z such that MM' = Sigma_Z. =#
if ishermitian(cor(x))
    Σ_Z = LinearAlgebra.cholesky(cor(x));
else
    Σ_Z = factorize(cor(x));
end
M = Σ_Z.L;
dim_M = size(M, 1);


Random.seed!(12345);


We probably should keep the rest of this within the policy loop

No we get into the weeds finally. 

* What should q_landing_probability be?

Try saving series of landing probabilities for use later...

In [16]:
q_landing_probability = landing_probability_load[!, :landing_probability];

Oh this is a 17520 vector... which comes from 365 * 48
* don't we need 8712 ? 
* or does lp_load already get the ones we need?

In [18]:
lp_load;

I think that if i just took the first half of this matrix it would be what i need.

In [23]:
left_half = lp_load[:, 1:size(lp_load, 2) ÷ 2];
q_landing_probability = vec(left_half);


now i need to find out which hour index we are in...

In [25]:
#=======================================================================
Determine length of Decision Problem
=======================================================================#
x = copy(wind_data);
# Sort data by issue time
sort!(x, :issue_time);
# Group data by issue time and count occurences in every group
df = combine(groupby(x, [:issue_time]), DataFrames.nrow => :count);
# Filter data by count. Only keep groups with 48 entries
df_filtered = filter(:count => ==(48), df);
issue_times_interest = df_filtered[!, :issue_time];
# find all forecast times for these issue times of interest
subset_wind_data = filter(row -> row[:issue_time] in issue_times_interest, wind_data);
subset_forecast_times = subset_wind_data[!, :forecast_time];
unique_forecast_times = unique(subset_forecast_times);
decision_T = length(unique_forecast_times);

unique_issue_times = unique(subset_wind_data[!, :issue_time]);

In [27]:
start_date = DateTime(string(scenario_year) * "-" * string(scenario_month) * "-" * string(scenario_day) * "T" * string(scenario_hour));

start_index = findfirst(isequal(start_date), unique_forecast_times)


5775

In [29]:
length(unique_forecast_times)

8736

In [ ]:
# find the forecast times associated with q_landing_probability

# print the forecast times to csv file for q_landing_probability and for unique_forecast_times


There are 24 more forecast times than we want can use right now... 
* where do they come from?
    * OH! they come from the last 24 hours where it is the 2DA forecast for the last available issue time.

- so then, we simply need to subtract the last 24 elements in unique forecast times to get the correct set...

In [10]:
# ==================================================================
# PROBABILITY GENERATION LOOP
# ==================================================================
#= In certain cases, as in solar power, not all columns will be 
useful. Some will be discarded to avoid problems in the factorization
of the correlation matrix. Here we check if the dimension n of the 
matrix M (n x n) is equal to the scenario length stipulated as 48.
If it is not, the vector W will have its length adjusted to n. 
The variable allequal_ind stores the indices of the columns that 
were discarded. After the scenarios Y are generated, Y column dimension
will be expanded and all-zero columns will be introduced in the 
location of the allequal_ind
=#
#Random.seed!(29031990)
Random.seed!(12345)
Y = Matrix{Float64}(undef, number_of_scenarios, scenario_length)

need_expansion = 0 # This is specially important for solar data with several columns whose st. dev. is zero
if dim_M != scenario_length
    original_scen_length = scenario_length
    scenario_length = dim_M
    Y = Matrix{Float64}(undef, number_of_scenarios, scenario_length)
    need_expansion = 1
end

for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        Y[nscen, k] = quantile(cdf_Z, q_landing_probability[start_index]);  
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

1000×48 Matrix{Float64}:
  3.1e-322       1.00277e-311  1.00275e-311  …  2.42281e-309  2.1186e-310
  1.00277e-311  -1.37294e-311  1.00277e-311     2.42824e-309  8.85467e-310
 -0.0            1.00277e-311  3.14493e-310     2.43367e-309  8.90899e-310
  3.14475e-310   6.9519e-310   3.14493e-310     2.43911e-309  8.96331e-310
  3.14475e-310   0.0           3.14492e-310     2.44454e-309  2.33589e-310
 -4.81734e-312   1.00277e-311  3.14493e-310  …  2.44997e-309  2.39022e-310
  1.00284e-311   0.0           3.14493e-310     2.28157e-310  9.12628e-310
  0.0            2.8362e-316   3.14493e-310     2.22725e-310  4.78044e-310
  1.0028e-311    0.0           3.14493e-310     2.17292e-310  4.61747e-310
  1.0037e-311    1.00277e-311  3.14493e-310     2.1186e-310   4.67179e-310
  ⋮                                          ⋱                
  1.00284e-311   1.00277e-311  7.29112e-304     8.31143e-310  2.77591e-309
  1.00277e-311   1.46e-321     0.0              8.36576e-310  2.78134e-309
  7.29112e-30

Test out quantile function

In [12]:
quantile(Normal(), 3/4)

0.6744897501960818